# Understand Text Similarity Space

`TextSimilaritySpace` creates the opportunity to embed text data to perform semantic search on top.

In [1]:
%pip install superlinked==36.3.1

In [2]:
import pandas as pd
from superlinked import framework as sl

pd.set_option("display.max_colwidth", 100)

In [3]:
class Paragraph(sl.Schema):
    id: sl.IdField
    body: sl.String


paragraph = Paragraph()

## Choosing your text embedding model

Currently we support [Sentence Transformers](https://www.sbert.net/docs/pretrained_models.html#model-overview) models for embedding textual data, as they are tailor-made for information retrieval on sentence long text.

Supply a [huggingface identifier](https://huggingface.co/sentence-transformers) of your choosing, and set the relevant SchemaField containing your text inputs. 

In [4]:
body_space = sl.TextSimilaritySpace(text=paragraph.body, model="sentence-transformers/all-mpnet-base-v2")
paragraph_index = sl.Index(body_space)

In [5]:
source: sl.InMemorySource = sl.InMemorySource(paragraph)
executor = sl.InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

In [6]:
source.put(
    [
        {"id": "paragraph-1", "body": "Glorious animals live in the wilderness."},
        {
            "id": "paragraph-2",
            "body": "Growing computation power enables advancements in AI.",
        },
    ]
)

In [7]:
query = sl.Query(paragraph_index).find(paragraph).similar(body_space, sl.Param("query_text")).select_all()

In [8]:
result = app.query(
    query,
    query_text="What makes the AI industry go forward?",
)
sl.PandasConverter.to_pandas(result)

,body,id,similarity_score
0,Growing computation power enables advancements in AI.,paragraph-2,0.462400
1,Glorious animals live in the wilderness.,paragraph-1,-0.040457
